In [1]:
import torch
import torch.nn as nn
import tiktoken

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [4]:
# example of unpacking:
primes = [2, 3, 5, 7, 11]
print(*primes)

2 3 5 7 11


In [5]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [7]:
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.4674, -0.5922,  0.9795,  ..., -1.2627,  0.6294, -0.5497],
         [ 1.2546,  0.2350, -0.0410,  ...,  1.4274,  0.2351, -0.9961],
         [ 1.7676,  0.7568,  2.2477,  ..., -0.6957, -0.2845,  1.1792],
         [ 0.2561, -0.8732,  0.3426,  ..., -0.3247, -1.2300, -0.3398]],

        [[ 0.0302, -0.7689,  0.8051,  ..., -0.9536,  0.8475, -0.8613],
         [ 0.5712,  1.5965, -0.1768,  ...,  0.7960, -0.2294, -1.1409],
         [ 2.1819,  0.5691,  0.8147,  ...,  0.5257,  0.8487,  0.4041],
         [ 0.3451, -0.3185, -0.6018,  ..., -0.1465, -0.8320, -0.4015]]],
       grad_fn=<UnsafeViewBackward0>)
